__Exploring Kepler’s Confirmed Planets Database__
==========================

Now that you have an understanding of stellar light curves and what we can learn from them, let’s explore some archived Kepler data for confirmed planets. To start, visit [this link](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS). 
This will bring you to the search page for the database of confirmed planets. We're going to explore this table and you can pick out your own exoplanet! 

To do this, follow these steps: 

1. In the column titled "**Planet Radius [Jupiter radius]**", type in  > 0.8. Note that this is in units of Jupiter radii. This value will ensure that the depth of our transit signal is deep and easy to detect.
2. In the column titled "**Orbital Period [days]**", let's make it less than 20 ays, so that it's more likely that you'll see multiple transits during your observation. For this, you have to type < 20.
3. Let's just look at planets that have been observed by Kepler, so in the "**Host Name**" column, write "Kepler". This way, we can go and readily look at the Kepler light curves which have all been made public.
4. Pick your exoplanet.  For the plots below in this notebook, I've choosen Kepler 435b as my example.
5. Clicking on the name will bring you to a page with more detailed information on your planet. You'll find  estimates quoted from the literature for various parameters, all of the publications of scientists who have studied this planet, and lots of other information.  Feel free to click "**Expand All**" in the header to expand all of the various tabs of information about this exoplanet.
6. Under the section called "**Kepler-#### Stellar Parameters**" (where #### would be replaced with the number ID for the exoplanet you chose), find the subsection called "**TOOLS**".  There are a lot of blue links in this section.  Click on the link that says "**Kepler TCE Time Series**", which will open a new tab in your browser.
7. In this new tab, you'll see a plot pop up that shows what may look like a band of color and then lots of random noise below it.  Click and drag with your mouse to zoom in (in the x direction) on a smaller part of the plot and you'll see that those points scattering down are actually individual exoplanet transits!  So what this plot is showing you is a very massive lightcurve spanning many, many days.
8. On the left side of the page, there's a panel that tells you what the x-axis column and y-axis column are.  Click on the "**Y Axis Column**" dropdown menu and select "**LC_DETREND [e-/e-]**", then click the "redraw" button at the top of that left panel.
9. Now to download your data!  On the bottom of the plot, there are a series of icons that let you play with the dimensions of the plot, how much you zoom in, etc.  Two of the icons have arrows pointing downwards, showing that they can be used to download information from the plot.  We want the download button on the right, that says "Download table of plotted data" if you hover over it with your mouse.  Once you've zoomed in on the plot to where you see 3-5 exoplanet transit dips only, click that download button and save the `.tbl` file somewhere on your computer (HINT: I recommend naming this file the name of the exoplanet you chose).
10. Awesome!  Now you can run the code below.
 
_________________________________________

## Fitting your *real* space telescope data

Okay!  So we've spent this entire week modifying and improving the way we make our `square_dip` function.  Now we get to use it to fit real Kepler Space Telescope data!

In [ ]:
import numpy as np # math package
import matplotlib.pyplot as plt # plotting package
import os # talks to your computer
import pandas as pd # works with tables of data

For this final round of modifying our `square_dip` function, because we'll be working with real data this time we can remove the `seed=88` optional input parameter.  Also, we've commented out the line were we randomly set the starting value for the first dip and INSTEAD we've added a `start` input parameter to the function.

This will help us fine tune where the first dip in real our data starts -- which is really important because that will set us up to fit the rest of the dips properly.

In [ ]:
def square_dip(t,array,mass,depth,period,start):
    '''
    t: The array for our x values (representing time)
    array: Our y-values! This allows us to have more
           than one planet :)
    mass: How large we want our planet to be!
          (Make this a number between 4 and 12)
    depth: How much should our planet dim the brightness?
           (Pick a value between 0.5 and 0.98)
    period: How frequently do we want this to happen?
    start: specifying the starting index!
    '''    
    
    transit_expected = array.copy()
    #start = round(np.random.uniform(1,20)) # instead of randomly choosing, we'll be specifying this
    stop = start + mass
    while stop < len(t):
        transit_expected[start:stop] = depth
        start += period
        stop = start + mass
    return transit_expected


Okay!  To upload our Kepler data, follow these steps: 

1. Find the file you downloaded from the Kepler archive that ends in `.tbl`. 
1. Click on the little folder icon on the left side of this notebook.
1. On the panel that pops out, notice that there are three icons underneath the header "**Files**".  Click on the left icon that looks like a page with and arrow pointing up.
1. Navigate the file browser that pops up to where you downloaded the `.tbl` file and double click on it to upload!

Once that's done, and you can see your `.tbl` in the left panel, we can run the code below.

In [ ]:
exoplanet = pd.read_csv("Kepler-435b.tbl", skiprows=3, delimiter='\s+',names=['time','lightcurve'])
exoplanet  # shows us what this table looks like

In [ ]:
time = exoplanet.time        # writing column to a variable
data = exoplanet.lightcurve  # writing column to a variable

In [ ]:
plt.figure(figsize=(20,5))
plt.scatter(time,data) # telling it to plot time on the x-axis and data on the y-axis

# we're going to fit these dips using our "square_dip" function
fit_time = np.linspace(time.min(),time.max(),len(time)) # just to make sure the data works in the fit
star = np.zeros(len(fit_time))

# play with the numerical inputs to the "square_dip" function to match the Kepler data!
planet = square_dip(fit_time,star,17,-0.0044,415,147)
plt.plot(fit_time,planet,color='r')

plt.xlabel('Time [days]') # add a name for the x-axis
plt.ylabel('Change in Brightness') # add a name for the y-axis

plt.show()